In [1]:
import pickle
import numpy as np
from re import split
data = pickle.load(open("data/emotype_v7.p", "rb" ))
print (data[0].keys())
print (len(data))

_, m = data[0]['encoding'].shape
n = len(data)
print (m)
print (n)

dict_keys(['text', 'label', 'prediction', 'encoding', 'outputs'])
52546
256
52546


In [2]:
label_cluster_dic = {'addiction':0, 'anxiety':1, 'autism':2, 
                         'bipolar':3, 'conversation':4, 'depression':5, 
                         'happy':6, 'schizophrenia':7}

In [3]:
def initialize_cluster_labels_with_ground_truth(data_dict):
    """Initialize dictionary of OPs with cluster labels according original topic. 
    """
    for i in range(len(data_dict)):
        data_dict[i]['cluster'] = label_cluster_dic[data_dict[i]['label']]
    return data_dict

In [4]:
data = initialize_cluster_labels_with_ground_truth(data)

In [5]:
data[2]['text']

"Hey guys, (backstory) I am 21, F and I have had GAD and Panic Disorder for about 56 years, about 4 years ago I started waking up feeling like I was barely rested in the first place. I have tried meds and I cannot tolerate any SSRI's so far or SNRI's. I try to take melatonin, tried to get to bed earlier, but no matter those changes or my anxiety level I never tend to rest enough (even in lower periods of anxiety). I also have constant body aches everyday for about 67 years. I have quit caffeine months ago because it stopped being enough to make me feel awakeit increased my anxiety. Another weird side note is an hour or two before bed I suddenly am feeling less fatigued.Any advice would be appreciated, Thanks.TLDR: I am constantly exhausted and I have GAD advice would be appreciated!"

In [6]:
len(data)

52546

A potentially smoother way to get a dictionary initialized with all the words present in the corpus...

In [7]:
# lexicon = set()
# for i in range(len(data)):
#     ith_phrase = data[i]['text'].strip().lower()  # Get text from ith OP
#     ith_phrase.replace("'", "")  # Get rid of apostrophes
#     list_of_words_in_phrase = split("[^a-zA-Z]+", ith_phrase)  
#     lexicon.update(list_of_words_in_phrase)  # Add all of the text's words to lexicon
# overall_word_freqs = {}.fromkeys(lexicon, 0)

In [8]:
# import random
# print(len(lexicon))
# some_words_in_the_lexicon = random.sample(lexicon, 10)  # Print 10 random words from our lexicon
# print(some_words_in_the_lexicon)
# print(len(overall_word_freqs.keys()))
# print(overall_word_freqs['i'])

In [9]:
all_text = []  # List to contain all of the words (our lexicon)
for i in range(len(data)):
    tmp = data[i]['text'].strip().lower()
    all_text.append(split("[^a-zA-Z']+", tmp))
print(len(all_text))
print(all_text[1])
dic = {}
for i in range(len(all_text)):  # i_th word in our lexicon
    for j in all_text[i]:  #j
        if j and dic.get(j, -1) != 0:
            dic[j] = 0
print(len(dic.keys()))
print(dic['i'])

52546
['i', 'just', 'had', 'an', 'accident', 'in', 'a', 'company', 'vehicle', 'i', 'scraped', 'the', 'left', 'wing', 'mirror', 'on', 'a', 'bush', 'driving', 'down', 'a', 'narrow', 'road', 'in', 'the', 'dark', 'the', 'mirror', 'was', 'intact', 'but', 'the', 'back', 'panel', 'of', 'the', 'mirror', 'came', 'off', 'this', 'was', 'a', 'brand', 'new', 'vehicle', 'and', 'tomorrow', "i'm", 'going', 'to', 'have', 'to', 'explain', 'what', 'happened', 'to', 'my', 'boss', 'this', "isn't", 'the', 'first', 'incident', "i've", 'been', 'in', 'either', 'i', "don't", 'even', 'want', 'to', 'drive', "it's", 'not', 'part', 'of', 'my', 'job', 'description', 'but', 'noone', 'else', 'will', 'do', 'it', 'noone', 'else', 'was', 'stupid', 'enough', 'i', 'guess', 'i', 'just', 'want', 'to', 'hide', 'in', 'a', 'corner', 'and', 'die', 'thinking', 'about', 'it', 'i', 'feel', 'like', 'such', 'a', 'useless', 'stupid', 'idiot', "it's", 'past', 'midnight', 'here', 'and', 'i', "can't", 'sleep', 'from', 'worry', '']
58236


In [10]:
word_list = list(dic.keys())
word_list[1]

"'im"

In [11]:
len(word_list)

58236

In [12]:
def find_text_with_cluster(cluster):
    out = []
    for i in range(len(data)):
        if data[i]['cluster'] == cluster:
            out.append(split("[^a-zA-Z']+", data[i]['text'].strip().lower() ))
    return out

def count_word_of_cluster(texts):
    out_dic = dic.fromkeys(dic,0)
    for text in texts:
        for word in text:
            if word: out_dic[word] += 1
    return out_dic

def get_cluster_vectors(word_list, dic_cluster):
    out = []
    for word in word_list:  
        out.append(dic_cluster[word])
    return out

def extract_vectors(cluster):
    text_cluster = find_text_with_cluster(cluster)
    dic_cluster = count_word_of_cluster(text_cluster)
    vector_cluster = get_cluster_vectors(word_list,dic_cluster)
    return vector_cluster

In [37]:
num_clusters = len(label_cluster_dic)  # We exlcude conversation
num_words_in_lexicon = len(word_list)

# Rows are clusters, columns are words, and elements are word frequencies in the cluster
word_freq_mat = np.zeros((num_clusters, num_words_in_lexicon))  
for cluster_name, cluster_label in label_cluster_dic.items():
    cluster_freqs = np.array(extract_vectors(cluster_label))
    word_freq_mat[cluster_label, :] = cluster_freqs

In [38]:
num_clusters_with_word_present = np.sum(word_freq_mat > 0, axis=0, keepdims=True)
tf_normalized_by_cluster = word_freq_mat / np.sum(word_freq_mat, axis=1, keepdims=True)
idf = np.log(num_clusters / num_clusters_with_word_present)

In [39]:
tfidf = tf_normalized_by_cluster * idf

In [40]:
n = 20
top_n_tfidf = np.argsort(tfidf, axis=1)[:, -n:]
bottom_n_tfidf = np.argsort(tfidf, axis=1)[:, :n]

In [41]:
[word_list[i] for i in list(top_n_tfidf[label_cluster_dic['autism'], :])]

['sensory',
 'pddnos',
 'meltdowns',
 'neurotypical',
 'stim',
 'workingout',
 'raspergers',
 'autistics',
 'nt',
 'stimming',
 'nonverbal',
 'asperger',
 "asperger's",
 'autistic',
 'nts',
 'asd',
 'autism',
 'aspergers',
 'aspie',
 'aspies']

In [50]:
[word_list[i] for i in list(top_n_tfidf[label_cluster_dic['bipolar'], :])]

['geodon',
 'diagnosis',
 'psychotic',
 'borderlines',
 'pdoc',
 'hypomanic',
 'hypo',
 'lithium',
 'abilify',
 'seroquel',
 'bp',
 'manic',
 'mania',
 'lamotrigine',
 'latuda',
 'dbt',
 'bpd',
 'hypomania',
 'fp',
 'lamictal']

In [52]:
[word_list[i] for i in list(top_n_tfidf[label_cluster_dic['depression'], :])]

['committing',
 'relapsed',
 'pills',
 'relapse',
 'razor',
 'cared',
 'loves',
 'numb',
 'killed',
 'sertraline',
 'pussy',
 'hurts',
 'empty',
 'harming',
 'selfharm',
 'killing',
 'alive',
 'pathetic',
 'worthless',
 'scars']

In [51]:
[word_list[i] for i in list(top_n_tfidf[label_cluster_dic['happy'], :])]

['graph',
 'testament',
 'achieving',
 'jeff',
 'rohn',
 'bennet',
 'beforeafter',
 'pomodoro',
 'materialization',
 'youtuberbettermentbookclub',
 'readout',
 'pz',
 "shrink's",
 'moviesvideos',
 'studybreak',
 'bennetcreated',
 'olsenvideos',
 'selfpost',
 'utcfriday',
 'utc']

# Ignore this bit, I was playing around with weightings...

In [ ]:
tf_normalized_over_all_clusters = np.sum(word_freq_mat, axis=0, keepdims=True) / np.sum(word_freq_mat)
np.min(tf_normalized_over_all_clusters)

In [43]:
eps = 1e-10
gamma = np.log((tf_normalized_by_cluster +  eps) / tf_normalized_over_all_clusters)

In [44]:
modified_tfidf = tfidf * gamma

In [45]:
print(np.min(tfidf))

0.0


In [46]:
print(np.max(tfidf))

0.00160351124941


In [47]:
n = 20
top_n_modified_tfidf = np.argsort(modified_tfidf, axis=1)[:, -n:]
bottom_n_modified_tfidf = np.argsort(modified_tfidf, axis=1)[:, :n]

In [49]:
[word_list[i] for i in list(top_n_modified_tfidf[label_cluster_dic['autism'], :])]

['sensory',
 'meltdowns',
 'neurotypical',
 'pddnos',
 'stim',
 'raspergers',
 'workingout',
 'autistics',
 'nt',
 'stimming',
 'nonverbal',
 'asperger',
 "asperger's",
 'autistic',
 'nts',
 'asd',
 'autism',
 'aspergers',
 'aspie',
 'aspies']